# BAMBOO: Binary descriptor based on AsymMetric pairwise BOOsting

In this notebook we include the implementation of the BAMBOO descriptor to provide a compressed representation of probe requests.

## Libraries and Configurations

Logger

In [133]:
import logging
from rich.logging import RichHandler

logging.getLogger("scapy.runtime").setLevel(logging.CRITICAL)

FORMAT = "%(message)s"
logging.basicConfig(
    level="NOTSET",
    format=FORMAT,
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)],
)

log = logging.getLogger("rich")

log.setLevel("CRITICAL")

Import configuration files

In [134]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

Import **data libraries**

In [135]:
import pandas as pd

Import **other libraries**

In [136]:
from rich.progress import Progress
from rich import traceback

traceback.install()

from tqdm.notebook import tqdm

In [137]:
import numpy as np
import math

Fancy print

In [138]:
class color:
    PURPLE = "\033[95m"
    CYAN = "\033[96m"
    DARKCYAN = "\033[36m"
    BLUE = "\033[94m"
    GREEN = "\033[92m"
    YELLOW = "\033[93m"
    RED = "\033[91m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"
    END = "\033[0m"

In [139]:
def bold_text(text: str) -> str:
    return str(color.BOLD + str(text) + color.END)

## Import Data

Importing **concatenated columns** and **pairs** datasets

In [140]:
pairs_df = pd.read_csv("../../data/interim/pairs_df.csv", index_col=0)

In [141]:
pairs_df

,Item 1,Item 2,Equality
0,0,1,1
1,0,2,1
2,0,3,1
3,0,4,1
4,0,5,1
...,...,...,...
11493610,4791,4793,1
11493611,4791,4794,1
11493612,4792,4793,1
11493613,4792,4794,1


In [142]:
strings_df = pd.read_csv("../../data/interim/string_df.csv", index_col=0)
strings_df = strings_df.rename(columns={strings_df.columns[0]: "Probes"})

In [143]:
strings_df

,Probes
0,0000000000000100000000100000010000001011000101...
1,0000000000000100000000100000010000001011000101...
2,0000000000000100000000100000010000001011000101...
3,0000000000000100000000100000010000001011000101...
4,0000000000000100000000100000010000001011000101...
...,...
4790,0000110000000100000000100000010000001011000101...
4791,0000101100000100000000100000010000001011000101...
4792,0001001000000100000000100000010000001011000101...
4793,0000100100000100000000100000010000001011000101...


Importing bitmask **filters**

In [144]:
filters_df = pd.read_csv("../../data/filters/bitmasks.csv", index_col=0)

In [145]:
print("Number of filters in the dataset: " + bold_text(filters_df.shape[0]))

Number of filters in the dataset: 65535


Select only filter corresponding to single columns (drop combinations).

In [146]:
filters_df = filters_df.head(16).reset_index()

Getting actual bitmask filters' column

In [147]:
filters = filters_df["Bitmask"]

Function to associate a **set of thresholds to each filter**, depending on the number of `1`.

In [148]:
def generate_thresholds(bitmasks):
    """
    Generate thresholds for each bitmask in a set.

    Parameters:
        bitmasks (set): A set containing the bitmasks.

    Returns:
        list: A list of tuples where each tuple contains a list of bitmasks sharing the same threshold list, and their corresponding threshold list.
    """
    threshold_dict = {}
    for bitmask in bitmasks:
        threshold = bitmask.count("1")
        if threshold in threshold_dict:
            threshold_dict[threshold].append(bitmask)
        else:
            threshold_dict[threshold] = [bitmask]

    return [
        (bitmasks, list(range(threshold + 1)))
        for threshold, bitmasks in threshold_dict.items()
    ]

**Generating thresholds** from bitmask filters

In [149]:
thresholds_list = generate_thresholds(filters)

In [150]:
print("Number of threshold sets: " + bold_text(len(thresholds_list)))

Number of threshold sets: 7


### Functions

The **bitwise AND** function performs said operation on 2 binary strings

In [151]:
def bitwise_and(bit_str1, bit_str2):
    # Convert bit strings to integers
    int1 = int(bit_str1, 2)
    int2 = int(bit_str2, 2)

    # Perform bitwise AND operation
    result = int1 & int2

    # Convert result back to binary string
    result_str = bin(result)[2:]  # [2:] to remove '0b' prefix

    # Return result
    return result_str.zfill(max(len(bit_str1), len(bit_str2)))

The **sum filter** takes as input a (binary) string and sums the values

In [152]:
def sumFilter(bitwise_and: str) -> int:
    sum = 0
    for i in bitwise_and:
        sum += int(i)
    return sum

**Sign function** returns -1 if negative value

In [153]:
def sign(number: int) -> int:
    if number < 0:
        return -1
    elif number >= 0:
        return 1

The **weak classifier** filters a couple of tuples, and given a threshold it, returns +1 or -1

In [154]:
def weak_classifier(pair: tuple, threshold: int, filter: str) -> int:
    log.debug(pair, threshold, filter)
    filtered1 = sumFilter(bitwise_and(pair[0], filter))
    filtered2 = sumFilter(bitwise_and(pair[1], filter))
    return sign((filtered1 - threshold) * (filtered2 - threshold))

Implementation of the **Dirach delta** function

In [155]:
def delta(prediction: int, ground_truth: int) -> int:
    if prediction != ground_truth:
        return 1
    else:
        return 0

The **get error** function calculates the weighted value of the filter, given the prevision and the ground truth

In [156]:
def get_error(weigth: float, prediction: int, ground_truth: int) -> float:
    error = weigth * delta(prediction, ground_truth)

    log.debug(f"Weigth {weigth}, Prediction {prediction}, Ground Truth {ground_truth}")

    return error

### BAMBOO

Input:
- Ground truth relationships $\langle x_{a(n)}, x_{b(n)}; y_n\rangle$
  - $n=1,..,N$
  - $y_n \in \{+1, -1\}$
- A set of filters $\mathcal{H} = \{h_1 , ..., h_F\}$
- A set of binarization thresholds $\mathcal{T} = \{t_1 , ..., t_T\}$

Output:
- A set of $M<F$ filters $[h_{i(1)}, ..., h_{i(M)}]$
- Corresponding set of binarization thresholds $[t_{j(1)}, ..., t_{j(M)}]$

Define **BAMBOO input**

In [157]:
# Input
dataset = strings_df.copy()
# pairs_index = pairs_df.copy()
pairs_index = pairs_df.head(100)
filters = thresholds_list
M = 3

# Initial weights
weights = np.ones(len(pairs_index)) / len(pairs_index)

# Errors per iteration
errors = {}

Algorithm implementation

In [158]:
for m in tqdm(range(M), desc="Iterations"):  # iterations
    for filters_entry in tqdm(filters, desc="Filters"):  # for each filter
        filters_list, threshold_list = filters_entry
        for filter, thresholds in tqdm(
            zip(filters_list, [threshold_list] * len(filters_list)),
            desc="Filter, Threshold zip",
            total=len(filters_list),
        ):
            for threshold in thresholds:  # for each threshold
                error = 0
                for pair in range(len(pairs_index)):  # for each pair
                    prediction = weak_classifier(
                        tuple(dataset.iloc[pairs_index.iloc[pair, 0:2], 0]),
                        threshold,
                        filter,
                    )
                    error += get_error(
                        weights[pair], prediction, pairs_index.iloc[pair, 2]
                    )
                    # print("LABEL", pairs_index.iloc[pair, 2])
                # print("[!] ERROR", error)
                errors[(filter, threshold)] = error
        # print("errors", errors)
    best_filter, best_threshold = min(errors, key=lambda k: abs(errors[k]))

    print("Best Filter:", best_filter)
    print("Best Threshold:", best_threshold)

    min_error = errors[(best_filter, best_threshold)]
    # print(min_error)
    if min_error == 0:
        min_error = 0.000000000000000000000001
    confidence = math.log(
        (1 - min_error) / min_error
    )  # confidence of the weak classifier
    # log.debug("Confidence:", confidence)

    # Asymmetric Weight Update
    for pair in range(len(pairs_index)):
        if pairs_index.iloc[pair, 2] == +1:
            if (
                weak_classifier(
                    tuple(dataset.iloc[pairs_index.iloc[pair, 0:2], 0]),
                    best_threshold,
                    best_filter,
                )
                != pairs_index.iloc[pair, 2]
            ):
                weights[pair] = weights[pair] * math.exp(confidence)

    for pair in range(len(pairs_index)):
        if pairs_index.iloc[pair, 2] == +1:
            weights[pair] = weights[pair] / sum(
                weights[pair]
                for pair in range(len(dataset))
                if pairs_index.iloc[pair, 2] == +1
            )

Iterations:   0%|          | 0/3 [00:00<?, ?it/s]

Filters:   0%|          | 0/7 [00:00<?, ?it/s]

Filter, Threshold zip:   0%|          | 0/9 [00:00<?, ?it/s]

Filter, Threshold zip:   0%|          | 0/2 [00:00<?, ?it/s]

Filter, Threshold zip:   0%|          | 0/1 [00:00<?, ?it/s]

...

Filter, Threshold zip:   0%|          | 0/1 [00:00<?, ?it/s]

Filter, Threshold zip:   0%|          | 0/1 [00:00<?, ?it/s]

Filter, Threshold zip:   0%|          | 0/1 [00:00<?, ?it/s]

Best Filter: 111111110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:53                                                                                   │
│                                                                                                  │
│   50 │                                                                                           │
│   51 │   for pair in range(len(dataset)):                                                        │
│   52 │   │   if pairs_index.iloc[pair, 2] == +1:                                                 │
│ ❱ 53 │   │   │   weights[pair] = weights[pair] / sum(                                            │
│   54 │   │   │   │   weights[pair]                                                               │
│   55 │   │   │   │   for pair in range(len(dataset))                                             │
│   56 │   │   │   │   if pairs_index.iloc[pair, 2] == +1                                          │
│                                                                                                  │
│ in <genexpr>:56                                                                                  │
│                                                                                                  │
│   53 │   │   │   weights[pair] = weights[pair] / sum(                                            │
│   54 │   │   │   │   weights[pair]                                                               │
│   55 │   │   │   │   for pair in range(len(dataset))                                             │
│ ❱ 56 │   │   │   │   if pairs_index.iloc[pair, 2] == +1                                          │
│   57 │   │   │   )                                                                               │
│   58                                                                                             │
│                                                                                                  │
│ /home/bacci/COMPACT/.venv/lib/python3.11/site-packages/pandas/core/indexing.py:1184 in           │
│ __getitem__                                                                                      │
│                                                                                                  │
│   1181 │   │   │   key = tuple(list(x) if is_iterator(x) else x for x in key)                    │
│   1182 │   │   │   key = tuple(com.apply_if_callable(x, self.obj) for x in key)                  │
│   1183 │   │   │   if self._is_scalar_access(key):                                               │
│ ❱ 1184 │   │   │   │   return self.obj._get_value(*key, takeable=self._takeable)                 │
│   1185 │   │   │   return self._getitem_tuple(key)                                               │
│   1186 │   │   else:                                                                             │
│   1187 │   │   │   # we by definition only have the 0th axis                                     │
│                                                                                                  │
│ /home/bacci/COMPACT/.venv/lib/python3.11/site-packages/pandas/core/frame.py:4200 in _get_value   │
│                                                                                                  │
│    4197 │   │   """                                                                              │
│    4198 │   │   if takeable:                                                                     │
│    4199 │   │   │   series = self._ixs(col, axis=1)                                              │
│ ❱  4200 │   │   │   return series._values[index]                                                 │
│    4201 │   │                                                                                    │
│    4202 │   │   series = self._get_item_cache(col)                                               │
│    4203 │   │   engine = self.index._engine                

In [ ]:
print("Best Filter:", best_filter)
print("Best Threshold:", best_threshold)
print("Min error", min_error)

Best Filter: 000000001111000000000000
Best Threshold: 2
Min error 0.0
